In [2]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import csv
#import tensorflow as tf

In [3]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [22]:
def readCSV2(path):
    f = open(path, 'rt')
    f.readline()
    for l in f:
        u,b = l.strip().split(',')
        yield u,b

In [58]:
allRatings = []
for l in readCSV("assignment1/train_Interactions.csv.gz"):
    allRatings.append(l)
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
items = set()
ratingDict = {}
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))
    ratingDict[(u,b)] = r
    usersPerItem[b].add(u)
    itemsPerUser[u].add(b)
    items.add(b)

In [4]:
numBooksRead = sorted([(len(v), k) for k, v in itemsPerUser.items()], reverse=True)

In [5]:
len(items)

6684

In [59]:
itemAverages = {}
    
for i, ls in ratingsPerItem.items():
    itemAverages[i] = sum([t[1] for t in ls]) / len(ls)

In [60]:
books = set()
users = set()
userBookRead = defaultdict(set)

for user, book, _ in allRatings:
    books.add(book)
    users.add(user)
    userBookRead[user].add(book)

In [61]:
negativeRatingsValid = []
for user, book, _ in ratingsValid:
    notRead = list(books.difference(userBookRead[user]))
    negativeSample = random.choice(notRead)
    negativeRatingsValid.append((user, negativeSample, 0))

In [62]:
for i in range(len(ratingsValid)):
    u,b,_ = ratingsValid[i]
    ratingsValid[i] = (u,b, 1)
validation = ratingsValid+negativeRatingsValid

In [63]:
validation[0]

('u59070515', 'b55084829', 1)

In [64]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

In [65]:
def Cosine(i1, i2):
    # Between two items
    try:
        inter = usersPerItem[i1].intersection(usersPerItem[i2])
        numer = 0
        denom1 = 0
        denom2 = 0
        for u in inter:
            numer += ratingDict[(u,i1)]*ratingDict[(u,i2)]
        for u in usersPerItem[i1]:
            denom1 += ratingDict[(u,i1)]**2
        for u in usersPerItem[i2]:
            denom2 += ratingDict[(u,i2)]**2
        denom = math.sqrt(denom1) * math.sqrt(denom2)
        if denom == 0: return 0
        return numer / denom
    except:
        return 0

In [66]:
def Pearson(i1, i2):
    try:
        # Between two items
        iBar1 = itemAverages[i1]
        iBar2 = itemAverages[i2]
        inter = usersPerItem[i1].intersection(usersPerItem[i2])
        numer = 0
        denom1 = 0
        denom2 = 0
        for u in inter:
            numer += (ratingDict[(u,i1)] - iBar1)*(ratingDict[(u,i2)] - iBar2)
        for u in inter: #usersPerItem[i1]:
            denom1 += (ratingDict[(u,i1)] - iBar1)**2
        #for u in usersPerItem[i2]:
            denom2 += (ratingDict[(u,i2)] - iBar2)**2
        denom = math.sqrt(denom1) * math.sqrt(denom2)
        if denom == 0: return 0
        return numer / denom
    except:
        return 0

In [67]:
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("assignment1/train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return2 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return2.add(i)
    if count > totalRead/1.4: break

In [104]:

def predict(user, book, trues):
    maxSim = 0
    for bPrime,_ in ratingsPerUser[user]:
        maxSim = max(maxSim, Cosine(book, bPrime))
    if trues > 10000:
        return 0
    elif threshold < maxSim or book in return2: 
        trues += 1
        return 1
    else:
        return 0

In [44]:
def predict2(user, book):
    maxSim = 0
    for bPrime,_ in ratingsPerUser[user]:
        maxSim = max(maxSim, Cosine(book, bPrime))
    if book in return2:
        maxSim += .5
    return maxSim

In [55]:
def predict3(user, book):
    return int(userBookPredictions[user][book])

In [45]:
threshold = .06
predictionsJ = []
for user,book,_ in validation:
    maxSim = 0
    for bPrime,_ in ratingsPerUser[user]:
        #maxSim = max(maxSim, Jaccard(usersPerItem[book], usersPerItem[bPrime]))
        #maxSim = max(maxSim, Pearson(book, bPrime))
        maxSim = max(maxSim, (Cosine(book, bPrime))) #+Jaccard(usersPerItem[book], usersPerItem[bPrime]))/2)#Jaccard(usersPerItem[book], usersPerItem[bPrime]))
    if truesPredictedPerUser[user] == truesExpectedPerUser[user]:
        predictionsJ.append(0)
    elif threshold < maxSim or book in return2: # or num of books users has read is greater than ?
        predictionsJ.append(1)
        truesPredictedPerUser[user] += 1
    else:
        predictionsJ.append(0)

In [57]:
len(test)

20000

In [107]:
userBookPredictions = defaultdict(dict)

In [108]:
for l in open("assignment1/pairs_Read.csv"):
    if l.startswith("userID"):
        continue
    u,b = l.strip().split(',')
    userBookPredictions[u][b] = predict2(u,b)

In [95]:
for user in userBookPredictions:
    sorted_books = sorted(userBookPredictions[user].items(), key=lambda x:x[1])

    for book,_ in sorted_books:
        if falseExpected[user] >0:
            userBookPredictions[user][book] = True
            falseExpected[user] -= 1
        else:
            userBookPredictions[user][book] = False


In [109]:
userBookPredictions

defaultdict(dict,
            {'u37758667': {'b99713185': 0.5, 'b05213070': 0},
             'u85626045': {'b00524816': 0.5,
              'b60770713': 0,
              'b29802159': 0.5,
              'b31024771': 0.5},
             'u70770448': {'b21349423': 0,
              'b77746740': 0.5,
              'b07327816': 0.5,
              'b92959743': 0},
             'u64714864': {'b52217488': 0.5,
              'b48279541': 0,
              'b06618138': 0,
              'b32857815': 0.5},
             'u78647159': {'b05439735': 0,
              'b63721105': 0.5,
              'b68809234': 0,
              'b00654647': 0},
             'u43398119': {'b08135061': 0.5, 'b51839247': 0.5},
             'u93156409': {'b24931555': 0, 'b24274613': 0.5},
             'u85724496': {'b15357546': 0.5, 'b39903678': 0.5},
             'u72905804': {'b31964420': 0, 'b78359727': 0.5},
             'u61280144': {'b93997659': 0.5, 'b05871767': 0},
             'u13480142': {'b20719123': 0, 'b48177176'

In [91]:
test = []
for l in open("assignment1/pairs_Read.csv"):
    if l.startswith("userID"):
        continue
    u,b = l.strip().split(',')
    test.append([u,b])

In [92]:
falseExpected = defaultdict(int)
users = set()

for u, _ in test:
    falseExpected[u] += 1
    users.add(u)
for u in falseExpected:
    falseExpected[u] = math.ceil(float(falseExpected[u])/2.0)

In [30]:
total = 0
for num in falseExpected.values():
    total += num
print(total)

10000


In [17]:
def accuracy(predictions, y):
    TP = sum([(p and l) for (p,l) in zip(predictions, y)])
    TN = sum([(not p and not l) for (p,l) in zip(predictions, y)])
    return (TP+TN)/len(predictions)

In [46]:
accuracy(predictionsJ, labelsValid)

0.82535

In [106]:
threshold = .06
trues = 0
with open("predictions_Read.csv", "w", newline='') as file:
    writer = csv.writer(file)
    for l in open("assignment1/pairs_Read.csv"):
        if l.startswith("userID"):
            writer.writerow(["userID", "bookID", "prediction"])
            continue
        u,b = l.strip().split(',')
        read = predict(u,b, trues)
        if read:
            trues += 1
            print("test")
        row = [u, b, read]
        writer.writerow(row)

test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test


test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test


test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test


test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test


test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test


In [27]:
# Q3

In [74]:
allRatings = []
for l in readCSV("assignment1/train_Interactions.csv.gz"):
    allRatings.append(l)

ratingsTrain = allRatings#[:190000]
#ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
usersPerItem = defaultdict(set)
userBiases = defaultdict(float)
itemBiases = defaultdict(float)
userIDs,itemIDs = {},{}
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))
    usersPerItem[b].add(u)
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not b in itemIDs: itemIDs[b] = len(itemIDs)

nUsers,nItems = len(userIDs),len(itemIDs)

In [55]:
ratings = set()
for _,_,r in ratingsTrain:
    ratings.add(r)
print(ratings)

{0, 1, 2, 3, 4, 5}


In [75]:
mean = sum([r for _,_,r in ratingsTrain])/len(ratingsTrain)
alpha = mean
N = len(ratingsTrain)
nUsers = len(ratingsPerUser)
nItems = len(ratingsPerItem)
users = list(ratingsPerUser.keys())
items = list(ratingsPerItem.keys())
userBiases = defaultdict(float)
itemBiases = defaultdict(float)

In [76]:
def prediction(user, item):
    if user in userBiases and item in itemBiases:
        num = alpha + userBiases[user] + itemBiases[item]
        if num > 5:
            return 5
        else: 
            return num
    elif user in userBiases:
        num = alpha + userBiases[user]
        if num > 5:
            return 5
        else: 
            return num
    elif item in itemBiases:
        num = alpha + itemBiases[item]
        if num > 5:
            return 5
        else: 
            return num
    else:
        return alpha 

In [77]:
for u in users:
    userBiases[u] = 0.0
for i in items:
    itemBiases[i] = 0.0
lamb = 4
for i in range(70):
    dalpha = 0
    for u,b,r in ratingsTrain:
        dalpha += r - (userBiases[u]+itemBiases[b])
    alpha = dalpha/N
    
    dUserBiases = defaultdict(float)
    for u in users:
        dUserBiases[u] = 0.0
    dItemBiases = defaultdict(float)
    for i in items:
        dItemBiases[u] = 0.0
        
    for user in userBiases:
        for book,rating in ratingsPerUser[user]:
            dUserBiases[user] += rating - (alpha+itemBiases[book])
        dUserBiases[user] = dUserBiases[user]/(lamb+len(ratingsPerUser[user]))
    userBiases = dUserBiases
    for item in itemBiases:
        for u, r in ratingsPerItem[item]:
            dItemBiases[item] += r - (alpha+userBiases[u])
        dItemBiases[item] = dItemBiases[item]/(lamb+len(ratingsPerItem[item]))
    itemBiases = dItemBiases

In [78]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [79]:
pred2 = [prediction(u,b) for u,b,_ in ratingsValid]
y = [r for _,_,r in ratingsValid]
validMSE = MSE(pred2,y)
validMSE

7.539727875228897

In [82]:

with open("predictions_Rating.csv", "w", newline='') as file:
    writer = csv.writer(file)
    for l in open("assignment1/pairs_Rating.csv"):
        if l.startswith("userID"):
            writer.writerow(["userID", "bookID", "prediction"])
            continue
        u,b = l.strip().split(',')
        row = [u, b, prediction(u,b)]
        writer.writerow(row)

In [167]:
userIDs = {}
itemIDs = {}
interactions = []
allRatings = []
for l in readCSV("assignment1/train_Interactions.csv.gz"):
    allRatings.append(l)
    
for u,i,r in allRatings:
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
    interactions.append((u,i,r))

In [168]:
random.shuffle(interactions)
len(interactions)

200000

In [169]:
nTrain = int(len(interactions) * 0.9)
nTest = len(interactions) - nTrain
interactionsTrain = interactions[:nTrain]
interactionsTest = interactions[nTrain:]

In [170]:
itemsPerUser = defaultdict(list)
usersPerItem = defaultdict(list)
for u,i,r in interactionsTrain:
    itemsPerUser[u].append(i)
    usersPerItem[i].append(u)

In [171]:
mu = sum([r for _,_,r in interactionsTrain]) / len(interactionsTrain)

In [172]:
optimizer = tf.keras.optimizers.Adam(0.1)

In [173]:
class LatentFactorModelBiasOnly(tf.keras.Model):
    def __init__(self, mu, lamb):
        super(LatentFactorModelBiasOnly, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.001))
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.lamb = lamb

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i]
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.betaU**2) +\
                            tf.reduce_sum(self.betaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        pred = self.alpha + beta_u + beta_i
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)

In [174]:
modelBiasOnly = LatentFactorModelBiasOnly(mu, 0.00001)

In [175]:
def trainingStepBiasOnly(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactions)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleR.append(r)

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
        (grad, var) in zip(gradients, model.trainable_variables)
        if grad is not None)
    return loss.numpy()

In [176]:
for i in range(50):
    obj = trainingStepBiasOnly(modelBiasOnly, interactionsTrain)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 0.6855031
iteration 20, objective = 0.65537083
iteration 30, objective = 0.6427225
iteration 40, objective = 0.6529196
iteration 50, objective = 0.6362116


In [177]:
biasPredictions =\
    [modelLFM.predict(userIDs[u], itemIDs[i]).numpy() for u,i,_ in interactionsTest]

In [178]:
labels = [r for _,_,r in interactionsTest]

In [179]:
MSE(biasPredictions, labels)

0.6319168756199247

In [185]:
def prediction(user, item):
    if user in userIDs and item in itemIDs:
        val = modelLFM.predict(userIDs[user], itemIDs[item]).numpy()
        if val <=5:
            return val
        else:
            return 5
    elif user in userBiases:
        val = alpha + userBiases[user]
        if val <=5:
            return val
        else:
            return 5
    elif item in itemBiases:
        val = alpha + itemBiases[item]
        if val <=5:
            return val
        else:
            return 5
    else:
        return alpha 

In [186]:
with open("predictions_Rating.csv", "w", newline='') as file:
    writer = csv.writer(file)
    for l in open("assignment1/pairs_Rating.csv"):
        if l.startswith("userID"):
            writer.writerow(["userID", "bookID", "prediction"])
            continue
        u,b = l.strip().split(',')
        row = [u, b, prediction(u,b)]
        writer.writerow(row)